In [2]:
import pandas as pd
import numpy as np
import datetime
taxi_df=pd.read_csv('C:/Users/91868/Desktop/taxi_fare_prediction.csv')

#need to delete the non impacting columns in the given dataset
taxi_df.drop(columns=['key'],inplace=True)

from sklearn.model_selection import train_test_split

y=taxi_df['fare_amount']
taxi_df.drop(columns='fare_amount',inplace=True)

#filling missing values
column=['dropoff_longitude','dropoff_latitude']
for column in taxi_df[column]:
    taxi_df[column].fillna(taxi_df[column].median(),inplace=True)

In [3]:
#feature engineering
feat_eng_data=taxi_df['pickup_datetime']

feat_eng_data

feat_eng_data=pd.to_datetime(feat_eng_data)

feat_eng_data.dt.hour

new_feature_df=pd.DataFrame({'Hour':feat_eng_data.dt.hour})

def time_of_day(hour):
    time=''
    if 0<=hour<=4:
        time=time.join('Midnight')
    elif 5<=hour<=10:
        time=time.join('Morning')
    elif 11<=hour<=16:
        time=time.join('Afternoon')
    elif 17<=hour<=20:
        time=time.join('Evening')
    elif 21<=hour<=23:
        time=time.join('Late Evening')
    return time

taxi_df['pickup_datetime']=new_feature_df['Hour'].apply(time_of_day)
#taxi_df['pickup_datetime'].replace((new_feature_df['Hour'].apply(time_of_day)),inplace=True)
#taxi_df['pickup_datetime']
taxi_df.rename(columns={'pickup_datetime':'Time_of_day'},inplace=True)

In [16]:
X_train,X_test,y_train,y_test=train_test_split(taxi_df,y,test_size=0.3,random_state=40)

catcols=[ele for ele in X_train.columns if X_train[ele].dtype==object]

from sklearn.preprocessing import LabelEncoder
for col in catcols:
    le=LabelEncoder()
    #le.fit(X_train[col])
    X_train[col]=le.fit_transform(X_train[col])
    X_test[col]=le.transform(X_test[col])
    
from sklearn.ensemble import RandomForestRegressor                   
rf_taxi_df=RandomForestRegressor(random_state=50,oob_score=True,min_samples_split=7,min_samples_leaf=8)

rf_taxi_df.fit(X_train,y_train)
                             
print("train_Score=",rf_taxi_df.score(X_train,y_train))
print("test_Score=",rf_taxi_df.score(X_test,y_test))

from sklearn.metrics import mean_squared_error

y_pred=rf_taxi_df.predict(X_test)
print(mean_squared_error(y_test,y_pred))

from math import sqrt
print(sqrt(mean_squared_error(y_test,y_pred)))

D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\ANACONDA - PYTHON\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ANACONDA - PYTHO

train_Score= 0.8359610350709443
test_Score= 0.7740263158140432
21.858232706622644
4.675278890785302


In [5]:
#for Determining the Feature Importance on the above Built RandomForestRegressor
importance=rf_taxi_df.feature_importances_
var_importance=pd.DataFrame(importance,index=X_train.columns,columns=['Importance'])
var_importance=var_importance.sort_values(by='Importance',ascending=False)
var_importance

,Importance
dropoff_longitude,0.466428
pickup_longitude,0.260421
dropoff_latitude,0.139819
pickup_latitude,0.124756
Time_of_day,0.006570
passenger_count,0.002006


In [ ]:
#to find the no. of Estimators(Trees) for a Random Forest
#above Program Used this 
import numpy as np
import matplotlib.pyplot as plt
estimators=np.arange(10,1000,50)
scores=[]
for n in estimators:
    rf_taxi_df.set_params(n_estimators=n)
    rf_taxi_df.fit(X_train,y_train)
    scores.append(rf_taxi_df.oob_score_)

#to Supress the Warnings
import warnings
warnings.filterwarnings("ignore")

df_scores=pd.DataFrame({'Scores':scores,'Number of Trees':estimators})
print(df_scores)
plt.title("Effect of n_estimators")
plt.xlabel("n_estimator")
plt.ylabel("score")
plt.plot(estimators,scores)

D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [8]:
from sklearn.model_selection import RandomizedSearchCV

grid1={'max_depth':[6,7,8,9,10],
       'min_samples_split':[3,4,5,6,7,8],
       'min_samples_leaf':[5,6,7,8,9]}

grid=RandomizedSearchCV(estimator=rf_taxi_df,param_distributions=grid1,n_iter=5,cv=5)
grid.fit(X_train,y_train)
grid.get_params
grid.cv_results_
print(grid.best_estimator_)
print(grid.best_score_)

grid.best_params_

D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\ANACONDA - PYTHON\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\A

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=7, min_samples_split=6,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=True, random_state=50, verbose=0,
                      warm_start=False)
0.7021254379093823


{'min_samples_split': 6, 'min_samples_leaf': 7, 'max_depth': 10}

In [14]:
taxi_df

,Time_of_day,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,Evening,-73.844311,40.721319,-73.841610,40.712278
1,Afternoon,-74.016048,40.711303,-73.979268,40.782004
2,Midnight,-73.982738,40.761270,-73.991242,40.750562
3,Midnight,-73.987130,40.733143,-73.991567,40.758092
4,Morning,-73.968095,40.768008,-73.956655,40.783762
5,Morning,-74.000964,40.731630,-73.972892,40.758233
6,Evening,-73.980002,40.751662,-73.973802,40.764842
7,Evening,-73.951300,40.774138,-73.990095,40.751048
8,Afternoon,-74.006462,40.726713,-73.993078,40.731628
9,Midnight,-73.980658,40.733873,-73.991540,40.758138


In [17]:
X_test

,Time_of_day,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
598221,0,-73.994650,40.755707,-73.986970,40.729437,5
221036,3,-73.967184,40.760170,-73.890126,40.747110,2
994273,0,-73.998463,40.760752,-73.999858,40.754888,1
995645,3,-74.000122,40.728043,-74.004440,40.716473,2
113974,0,-73.988572,40.748015,-73.998998,40.684937,1
231026,1,-73.978494,40.752801,-73.990722,40.753664,1
395746,4,-73.957110,40.783172,-73.960495,40.781407,3
283100,3,-73.988140,40.749110,-73.972920,40.758127,1
770216,1,-73.782095,40.644747,-73.981063,40.680500,3
259909,1,-73.981828,40.766756,-73.988993,40.756597,1
